# STAC API queries with Python and httpx

This notebook aims to demonstrate how [eoAPI](https://eoapi.dev) can be used to analyze Maxar's high-resolution satellite data to assess the Kahramanmaras earthquakes' impact.

**Requirements**: `httpx ipyleaflet`

### Maxar Open Data

Pre and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. These images are generated using the Maxar ARD pipeline, tiled on an organized grid in analysis-ready cloud-optimized formats.

### STAC and COGs

Maxar releases open data for select sudden-onset major crisis events. In addition to making the data (as nice COGs) freely available on AWS, they also add STAC (static) metadata alongside the images. Having the STAC items already created makes ingestion into the PgSTAC database easy (we don't have to produce the items ourselves and thus have to read the images).

To learn more about ingesting the Maxar OpenData STAC catalog into PgSTAC see https://github.com/vincentsarago/MAXAR_opendata_to_pgstac.


In [ ]:
!python -m pip install httpx ipyleaflet

In [2]:
stac_endpoint = "https://stac.eoapi.dev"

### Collection

If you look in `https://stac.eoapi.dev/collections` response, you'll find one collection for the Kahramanmaras earthquake named `MAXAR_Kahramanmaras_turkey_earthquake_23`.

In [3]:
import json
from datetime import datetime

import httpx
import ipyleaflet

In [4]:
# list the collections and find the collection_id associated to the `kahramanmaras` event
collections = httpx.get(f"{stac_endpoint}/collections").json()
collection_names = ([c["id"] for c in collections["collections"]])

print(f"Number of collections: {len(collection_names)}")
print(f"Collection names: {collection_names}")

Number of collections: 35
Collection names: ['MAXAR_BayofBengal_Cyclone_Mocha_May_23', 'MAXAR_New_Zealand_Flooding22', 'MAXAR_ghana_explosion22', 'MAXAR_kentucky_flooding_7_29_2022', 'UMBRA_2023', 'MAXAR_Emilia_Romagna_Italy_flooding_may23', 'MAXAR_Gambia_flooding_8_11_2022', 'MAXAR_Hurricane_Fiona_9_19_2022', 'MAXAR_Hurricane_Ian_9_26_2022', 'MAXAR_Hurricane_Idalia_Florida_Aug23', 'MAXAR_India_Floods_Oct_2023', 'MAXAR_Indonesia_Earthquake22', 'MAXAR_Kahramanmaras_turkey_earthquake_23', 'MAXAR_Kalehe_DRC_Flooding_5_8_23', 'MAXAR_Libya_Floods_Sept_2023', 'MAXAR_Marshall_Fire_21_Update', 'MAXAR_Maui_Hawaii_fires_Aug_23', 'MAXAR_McDougallCreekWildfire_BC_Canada_Aug_23', 'MAXAR_Morocco_Earthquake_Sept_2023', 'MAXAR_NWT_Canada_Aug_23', 'MAXAR_Nepal_Earthquake_Nov_2023', 'MAXAR_New_Zealand_Flooding23', 'MAXAR_Sudan_flooding_8_22_2022', 'MAXAR_afghanistan_earthquake22', 'MAXAR_cyclone_emnati22', 'MAXAR_pakistan_flooding22', 'MAXAR_shovi_georgia_landslide_8Aug23', 'MAXAR_southafrica_flooding22

Lest focus on the data acquired for the M7.8 and M7.5 Kahramanmaras earthquakes in Turkey on February 6, 2023.

More on the event: https://www.usgs.gov/news/featured-story/m78-and-m75-kahramanmaras-earthquake-sequence-near-nurdagi-turkey-turkiye


Collection's Name: **MAXAR_Kahramanmaras_turkey_earthquake_23**


Let's check the collection's metadata:

In [5]:
collection_id = "MAXAR_Kahramanmaras_turkey_earthquake_23"

collection_info = httpx.get(f"{stac_endpoint}/collections/{collection_id}").json()

print(collection_info)

geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [bbox[0], bbox[1]],
                    [bbox[2], bbox[1]],
                    [bbox[2], bbox[3]],
                    [bbox[0], bbox[3]],
                    [bbox[0], bbox[1]],
                ]]
            },
            'properties': {}
        }
        for bbox in collection_info["extent"]["spatial"]["bbox"]
    ]
}

mainbbox = collection_info["extent"]["spatial"]["bbox"][0]

m = ipyleaflet.leaflet.Map(
    center=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom=7
)

geo_json = ipyleaflet.leaflet.GeoJSON(data=geojson)
m.add_layer(geo_json)
m

{'id': 'MAXAR_Kahramanmaras_turkey_earthquake_23', 'type': 'Collection', 'links': [{'rel': 'items', 'type': 'application/geo+json', 'href': 'https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23/items'}, {'rel': 'parent', 'type': 'application/json', 'href': 'https://stac.eoapi.dev/'}, {'rel': 'root', 'type': 'application/json', 'href': 'https://stac.eoapi.dev/'}, {'rel': 'self', 'type': 'application/json', 'href': 'https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23'}], 'title': 'Turkey and Syria Earthquake 2023', 'extent': {'spatial': {'bbox': [[35.302597, 35.875122, 40.310497, 38.47292570695286], [37.2976, 36.98959965805714, 37.47444448907068, 37.015901889979396], [37.29774464331677, 36.9896650792383, 37.457524034308584, 37.117494], [36.141868, 36.26635032674398, 36.244766, 36.393881], [36.14285540815239, 36.27356734921313, 36.244766, 36.393881], [36.902291, 37.557316, 36.958307, 37.596933], [36.624329, 36.345093, 36.735901, 36.434875], [

Map(center=[37.17402385347643, 37.806546999999995], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Each collection can have spatial and temporal extents. As for the spatial extent, a collection can have multiple temporal extents, but its first one represents the combined min/max of all the intervals.

In [6]:
print(collection_info["extent"]["temporal"])

{'interval': [['2021-02-28T08:10:22Z', '2023-03-11T08:29:15Z']]}


## Items

In this section, we will:

- List all items for a specific collection using the `/collections/{collection_id}/items` endpoint
- Talk about the `limit` parameter
- Visualize all items on a map
- Talk about the item metadata

- List the Assets available for one Item

In [7]:
items = httpx.get(f"{stac_endpoint}/collections/{collection_id}/items").json()


print(f"Nb Items in Db: {items['context']['matched']}")  # This is only available if CONTEXT=ON
print(f"Returned {len(items['features'])} Items")

Nb Items in Db: 2115
Returned 10 Items


As you can see below, the `/items` endpoints returned only 10 items. To return more data, we need to either use the `paging` mechanism.

In [8]:
kahramanmaras_items = []

url = f"{stac_endpoint}/collections/{collection_id}/items"
while True:
    items = httpx.get(url, params={"limit": 100}).json()
    
    kahramanmaras_items.extend(items["features"])
    next_link = list(filter(lambda link: link["rel"] == "next", items["links"]))
    if next_link:
        url = next_link[0]["href"]
    else:
        break

print(f"Nb Items: {len(kahramanmaras_items)}")

Nb Items: 2115


In [9]:
m = ipyleaflet.leaflet.Map(
    center=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom=7
)

event_date = datetime(2023, 2, 6, hour=0, minute=0)

# Use a styling function to show where we have before/after items
def style_function(feature):
    d = datetime.strptime(feature["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ")
    return {
        "fillOpacity": 0.1,
        "weight": 0.1,
        # Blue for pre-event / red for post-event
        "fillColor": "#0000ff" if d < event_date else "#ff0000"
    }

geo_json = ipyleaflet.leaflet.GeoJSON(data={"type": "FeatureCollection", "features": kahramanmaras_items}, style_callback=style_function)
m.add_layer(geo_json)
m

Map(center=[37.17402385347643, 37.806546999999995], controls=(ZoomControl(options=['position', 'zoom_in_text',…

#####  Item metadata 

Each item should have an `id`, a `geometry`, some links to `Assets`, and a set of properties.

Item specification: https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md

In [10]:
item = kahramanmaras_items[0]
print("Item example:")
print(json.dumps(item, indent=4))

Item example:
{
    "id": "37_031133210001_10300100E49E8000",
    "bbox": [
        36.387245559111236,
        36.092041667396806,
        36.42079643694594,
        36.11834923545021
    ],
    "type": "Feature",
    "links": [
        {
            "rel": "collection",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23"
        },
        {
            "rel": "parent",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23"
        },
        {
            "rel": "root",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/"
        },
        {
            "rel": "self",
            "type": "application/geo+json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23/items/37_031133210001_10300100E49E8000"
        }
    ],
    "assets": {
     

In [11]:
print("Item Id", item["id"])
print("Item Assets:", list(item["assets"].keys()))
print("Item Properties:")
print(json.dumps(item["properties"], indent=4))

Item Id 37_031133210001_10300100E49E8000
Item Assets: ['visual', 'data-mask', 'cloud-mask', 'ms_analytic', 'pan_analytic', 'cloud-mask-raster', 'cloud-shadow-mask', 'building-centroids', 'building-footprints']
Item Properties:
{
    "gsd": 0.48,
    "quadkey": "031133210001",
    "datetime": "2023-03-11T08:29:15Z",
    "platform": "WV02",
    "utm_zone": 37,
    "grid:code": "MXRA-Z37-031133210001",
    "proj:bbox": [
        264843.75,
        3997237.548828125,
        267866.2109375,
        4000156.25
    ],
    "proj:epsg": 32637,
    "catalog_id": "10300100E49E8000",
    "view:azimuth": 175.5,
    "proj:geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    264843.75,
                    4000156.25
                ],
                [
                    264843.75,
                    3997362.6708984375
                ],
                [
                    266686.70654296875,
                    3997294.921875
     

#### Find acquisition times

Every item should have either a `datetime` or a `start/end_datetime` property. For the Maxar dataset, we are assuming that `datetime` is acquisition times.

In [12]:
datetimes = {item["properties"]["datetime"] for item in kahramanmaras_items}
print("Dates:", sorted(list(datetimes))[0:10])

Dates: ['2021-02-28T08:10:22Z', '2021-02-28T08:10:23Z', '2021-08-17T11:16:54Z', '2021-08-18T08:13:11Z', '2021-08-18T08:13:12Z', '2021-09-09T08:36:18Z', '2021-09-09T08:36:19Z', '2021-09-09T08:36:20Z', '2021-09-27T08:40:11Z', '2021-09-27T08:40:12Z']


Let's sort the items in two categories: `before` and `after` the event.

In [13]:
event_date = datetime(2023, 2, 6, hour=0, minute=0)

pre_items = list(
    filter(
        lambda item: datetime.strptime(item["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ") < event_date, 
        kahramanmaras_items
    )
)

post_items = list(
    filter(
        lambda item: datetime.strptime(item["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ") >= event_date, 
        kahramanmaras_items
    )
)
print("PRE event items:", len(pre_items))
print("POST event items:", len(post_items))

PRE event items: 229
POST event items: 1886


In [14]:
# Same but using the STAC API
pre_items_api = httpx.post(
    f"{stac_endpoint}/search",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "in", 
                        "args": [{"property": "collection"}, [collection_id]]
                    },
                    {
                        "op": "lt", 
                        "args": [
                            {"property": "datetime"}, "2023-02-06T00:00:00Z"
                        ]
                    }
                ],
            },
        }
    )
).json()

print(f"Nb Items in Db: {pre_items_api['context']['matched']}")  # This is only available if CONTEXT=ON

Nb Items in Db: 229


## Asset visualization

So we have **2115** items for the `MAXAR_Kahramanmaras_turkey_earthquake_23` collection, and each item has **4** assets (this is also found at the collection level in the `item_assets extension`).